In [ ]:
%matplotlib inline

import re

import pickle
import collections
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from utils import get_wandb_logs, num_params, pd, clean_model_names

In [ ]:

wandb_path_list = [
    "Timm_OfficeHome_ERM_momentum_sgd_v11",
    "Timm_PACS_ERM_momentum_sgd_v11",
    "Timm_DomainNet_ERM_momentum_sgd_v11",
    "Timm_VLCS_3_ERM_momentum_sgd_v11",
]

# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v10",
#     "Timm_PACS_ERM_momentum_sgd_v10",
#     "Timm_DomainNet_ERM_momentum_sgd_v10",
#     "Timm_VLCS_3_ERM_momentum_sgd_v10",
# ]

# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v9",
#     "Timm_PACS_ERM_momentum_sgd_v9",
#     "Timm_DomainNet_ERM_momentum_sgd_v9",
#     "Timm_VLCS_3_ERM_momentum_sgd_v9",
# ]

# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v8",
#     "Timm_PACS_ERM_momentum_sgd_v8",
#     "Timm_DomainNet_ERM_momentum_sgd_v8",
#     "Timm_VLCS_3_ERM_momentum_sgd_v8",
# ]


# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v7",
#     "Timm_PACS_ERM_momentum_sgd_v7",
#     "Timm_DomainNet_ERM_momentum_sgd_v7",
#     "Timm_VLCS_3_ERM_momentum_sgd_v7",
# ]


# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v6",
#     "Timm_PACS_ERM_momentum_sgd_v6",
#     "Timm_DomainNet_ERM_momentum_sgd_v6",
#     "Timm_VLCS_3_ERM_momentum_sgd_v6",
# ]

# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v5",
#     "Timm_PACS_ERM_momentum_sgd_v5",
#     "Timm_DomainNet_ERM_momentum_sgd_v5",
#     "Timm_VLCS_3_ERM_momentum_sgd_v5",
# ]

# wandb_path_list = [
#     "Timm_OfficeHome_ERM_momentum_sgd_v4",
#     "Timm_PACS_ERM_momentum_sgd_v4",
#     "Timm_DomainNet_ERM_momentum_sgd_v4",
#     "Timm_VLCS_3_ERM_momentum_sgd_v4",
# ]

In [ ]:
_dfs = {}
entity = "anonymized"
for wandb_path in wandb_path_list:
    tmp_df  = get_wandb_logs(f"{entity}/{wandb_path}")
    _dfs[f'{wandb_path}'] = tmp_df

In [ ]:
def print_shell_args(df, wandb_project_name):

    print(f"\n========================================================================\n")
    print(f"wandb_project_name: {wandb_project_name}\n")

    # Grouping by model and getting the row with the maximum avg_val_acc for each model
    best_lrs = df.groupby("model", group_keys=False).apply(lambda group: group.nlargest(1, "avg_val_acc"))

    # Extracting lr, avg_val_acc, and avg_test_acc for each model with the highest avg_val_acc
    best_metrics = best_lrs[["model", "lr", "avg_val_acc", "avg_test_acc"]]

    print("Best Metrics:\n", best_metrics, "\n")

    shell_commands = []

    for _, row in best_metrics.iterrows():
        command = f"MODEL_LR_MAP['{row['model']}']={row['lr']:.4f}"
        shell_commands.append(command)

    shell_commands_output = '\n'.join(shell_commands)

    print("\nShell Commands:\n")
    print(shell_commands_output)

In [ ]:
for wandb_project_name in wandb_path_list:
    df = _dfs[wandb_project_name]
    print_shell_args(df, wandb_project_name)